In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
import numpy as np

model_name = '/var/projetos/Jupyterhubstorage/victor.silva/brazilian-legal-text-bert/Modelos/epocas_HelBERT-base-uncased/checkpoint-epoca-30'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def score(model, tokenizer, sentence):
    tensor_input = tokenizer.encode(sentence, return_tensors='pt', truncation=True, max_length=128)
    repeat_input = tensor_input.repeat(tensor_input.size(-1)-2, 1)
    mask = torch.ones(tensor_input.size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())

print(score(sentence='Os furos das luminárias existentes deverão ser fechadas com argamassa e gesso. alkdjaklsjd asdlkjaslkdj adkljaklsdj asdlkjaskld asdklj klasdj asdjl adskjl.', model=model, tokenizer=tokenizer))

In [ ]:
!pip install evaluate

In [ ]:
from evaluate import load
perplexity = load("perplexity", module_type="metric")
input_texts = ["O objeto da licitação é a compra de comida."]
results = perplexity.compute(model_id='/var/projetos/Jupyterhubstorage/victor.silva/brazilian-legal-text-bert/Modelos/epocas_HelBERT-base-uncased/checkpoint-epoca-30',
                             add_start_token=False,
                             predictions=input_texts)
print(results)

In [ ]:
import pandas as pd

In [ ]:
with open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/PreTreinamento/HelBERT-base-uncased/teste.txt') as f:
    lines = [line.strip() for line in f.readlines()]

In [ ]:
dados = pd.DataFrame(lines, columns=['text'])
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Perplexidade/perplexidade_teste.csv', index=False)

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Perplexidade/LipSET.csv')
dados

In [ ]:
from nltk import sent_tokenize

In [ ]:
novas_sentencas = []
for indice in dados.index:
    frases = sent_tokenize(dados.loc[indice, 'text'])
    novas_sentencas.extend(frases)

In [ ]:
dados = pd.DataFrame(novas_sentencas, columns=['text'])
dados

In [ ]:
dados['tamanho'] = dados['text'].apply(lambda x: len(x.split()))

In [ ]:
dados['tamanho'].describe()

In [ ]:
dados = dados[dados['tamanho'] >= 10]
dados = dados[dados['tamanho'] <= 64]
dados

In [ ]:
dados.drop_duplicates(inplace=True)
dados.dropna(inplace=True)
dados.reset_index(drop=True, inplace=True)
dados.drop('tamanho', axis=1, inplace=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Perplexidade/LipSET-reduzida.csv', index=False)